In [2]:
import pandas as pd
import numpy as np
from datetime import date
import numpy_financial as npf

In [3]:
def amortization_table(interest_rate, years, payments_year, principal, addl_principal=0, start_date=date.today()):
    """ Calculate the amortization schedule given the loan details
    
     Args:
        interest_rate: The annual interest rate for this loan
        years: Number of years for the loan
        payments_year: Number of payments in a year
        principal: Amount borrowed
        addl_principal (optional): Additional payments to be made each period. Assume 0 if nothing provided.
                                   must be a value less then 0, the function will convert a positive value to
                                   negative
        start_date (optional): Start date. Will start on first of next month if none provided

    Returns:
        schedule: Amortization schedule as a pandas dataframe
        summary: Pandas dataframe that summarizes the payoff information
    """
    # Ensure the additional payments are negative
    if addl_principal > 0:
        addl_principal = -addl_principal
    
    # Create an index of the payment dates
    rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
    rng.name = "Payment_Date"
    
    # Build up the Amortization schedule as a DataFrame
    df = pd.DataFrame(index=rng,columns=['Payment', 'Principal', 'Interest', 
                                         'Addl_Principal', 'Curr_Balance'], dtype='float')
    
    # Add index by period (start at 1 not 0)
    df.reset_index(inplace=True)
    df.index += 1
    df.index.name = "Period"
    
    # Calculate the payment, principal and interests amounts using built in Numpy functions
    per_payment = npf.pmt(interest_rate/payments_year, years*payments_year, principal)
    df["Payment"] = per_payment
    df["Principal"] = npf.ppmt(interest_rate/payments_year, df.index, years*payments_year, principal)
    df["Interest"] = npf.ipmt(interest_rate/payments_year, df.index, years*payments_year, principal)
        
    # Round the values
    df = df.round(2) 
    
    # Add in the additional principal payments
    df["Addl_Principal"] = addl_principal
    
    # Store the Cumulative Principal Payments and ensure it never gets larger than the original principal
    df["Cumulative_Principal"] = (df["Principal"] + df["Addl_Principal"]).cumsum()
    df["Cumulative_Principal"] = df["Cumulative_Principal"].clip(lower=-principal)
    
    # Calculate the current balance for each period
    df["Curr_Balance"] = principal + df["Cumulative_Principal"]
    
    # Determine the last payment date
    try:
        last_payment = df.query("Curr_Balance <= 0")["Curr_Balance"].idxmax(axis=1, skipna=True)
    except ValueError:
        last_payment = df.last_valid_index()
    
    last_payment_date = "{:%m-%d-%Y}".format(df.loc[last_payment, "Payment_Date"])
        
    # Truncate the data frame if we have additional principal payments:
    if addl_principal != 0:
                
        # Remove the extra payment periods
        df = df.loc[0:last_payment].copy()
        
        # Calculate the principal for the last row
        df.loc[last_payment, "Principal"] = -(df.loc[last_payment-1, "Curr_Balance"])
        
        # Calculate the total payment for the last row
        df.loc[last_payment, "Payment"] = df.loc[last_payment, ["Principal", "Interest"]].sum()
        
        # Zero out the additional principal
        df.loc[last_payment, "Addl_Principal"] = 0
        
    # Get the payment info into a DataFrame in column order
    payment_info = (df[["Payment", "Principal", "Addl_Principal", "Interest"]]
                    .sum().to_frame().T)
       
    # Format the Date DataFrame
    payment_details = pd.DataFrame.from_dict(dict([('payoff_date', [last_payment_date]),
                                               ('Interest Rate', [interest_rate]),
                                               ('Number of years', [years])
                                              ]))
    # Add a column showing how much we pay each period.
    # Combine addl principal with principal for total payment
    payment_details["Period_Payment"] = round(per_payment, 2) + addl_principal
    
    payment_summary = pd.concat([payment_details, payment_info], axis=1)
    return df, payment_summary
    

# 20 year amortization schedule

In [13]:
interest_rate = 0.025
years = 20
payments_year = 12 
principal = 1000000

In [14]:
schedule1, stats1 = amortization_table(interest_rate, years, payments_year, principal)

## interest amount paid every month

In [16]:
print(list(schedule1.Interest)) # Payment_Date 	Payment 	Principal 	Interest 	Addl_Principal 	Curr_Balance 	Cumulative_Principal

[-2083.33, -2076.63, -2069.92, -2063.19, -2056.45, -2049.7, -2042.93, -2036.14, -2029.35, -2022.53, -2015.71, -2008.87, -2002.01, -1995.14, -1988.26, -1981.36, -1974.45, -1967.53, -1960.59, -1953.63, -1946.66, -1939.68, -1932.68, -1925.66, -1918.64, -1911.59, -1904.54, -1897.47, -1890.38, -1883.28, -1876.16, -1869.03, -1861.88, -1854.72, -1847.55, -1840.36, -1833.15, -1825.93, -1818.7, -1811.45, -1804.18, -1796.9, -1789.6, -1782.29, -1774.96, -1767.62, -1760.27, -1752.89, -1745.51, -1738.1, -1730.68, -1723.25, -1715.8, -1708.33, -1700.85, -1693.36, -1685.85, -1678.32, -1670.78, -1663.22, -1655.64, -1648.05, -1640.45, -1632.82, -1625.19, -1617.53, -1609.86, -1602.18, -1594.47, -1586.76, -1579.02, -1571.27, -1563.51, -1555.72, -1547.93, -1540.11, -1532.28, -1524.43, -1516.57, -1508.69, -1500.79, -1492.88, -1484.95, -1477.0, -1469.04, -1461.06, -1453.07, -1445.05, -1437.02, -1428.98, -1420.92, -1412.84, -1404.74, -1396.63, -1388.5, -1380.35, -1372.19, -1364.01, -1355.81, -1347.59, -1339.3

## monthly payment

In [17]:
print(list(schedule1.Payment))

[-5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03, -5299.03,

## monthly principal paid

In [18]:
print(list(schedule1.Principal))

[-3215.7, -3222.39, -3229.11, -3235.84, -3242.58, -3249.33, -3256.1, -3262.89, -3269.68, -3276.49, -3283.32, -3290.16, -3297.02, -3303.88, -3310.77, -3317.66, -3324.58, -3331.5, -3338.44, -3345.4, -3352.37, -3359.35, -3366.35, -3373.36, -3380.39, -3387.43, -3394.49, -3401.56, -3408.65, -3415.75, -3422.87, -3430.0, -3437.14, -3444.31, -3451.48, -3458.67, -3465.88, -3473.1, -3480.33, -3487.58, -3494.85, -3502.13, -3509.43, -3516.74, -3524.06, -3531.41, -3538.76, -3546.14, -3553.52, -3560.93, -3568.35, -3575.78, -3583.23, -3590.69, -3598.17, -3605.67, -3613.18, -3620.71, -3628.25, -3635.81, -3643.39, -3650.98, -3658.58, -3666.21, -3673.84, -3681.5, -3689.17, -3696.85, -3704.55, -3712.27, -3720.01, -3727.76, -3735.52, -3743.3, -3751.1, -3758.92, -3766.75, -3774.6, -3782.46, -3790.34, -3798.24, -3806.15, -3814.08, -3822.03, -3829.99, -3837.97, -3845.96, -3853.98, -3862.0, -3870.05, -3878.11, -3886.19, -3894.29, -3902.4, -3910.53, -3918.68, -3926.84, -3935.02, -3943.22, -3951.44, -3959.67, -

In [19]:
print(list(schedule1.Curr_Balance))

[996784.3, 993561.91, 990332.8, 987096.96, 983854.38, 980605.05, 977348.95, 974086.06, 970816.38, 967539.89, 964256.57, 960966.41, 957669.39, 954365.51, 951054.74, 947737.0800000001, 944412.5, 941081.0, 937742.56, 934397.16, 931044.79, 927685.4400000001, 924319.09, 920945.73, 917565.34, 914177.91, 910783.42, 907381.86, 903973.21, 900557.46, 897134.5900000001, 893704.5900000001, 890267.4500000001, 886823.14, 883371.66, 879912.99, 876447.11, 872974.01, 869493.68, 866006.1000000001, 862511.25, 859009.12, 855499.6900000001, 851982.9500000001, 848458.89, 844927.48, 841388.72, 837842.5800000001, 834289.06, 830728.13, 827159.78, 823584.0, 820000.77, 816410.0800000001, 812811.91, 809206.24, 805593.06, 801972.35, 798344.1, 794708.29, 791064.9, 787413.9199999999, 783755.34, 780089.13, 776415.29, 772733.79, 769044.62, 765347.77, 761643.22, 757930.95, 754210.94, 750483.1799999999, 746747.66, 743004.36, 739253.26, 735494.3400000001, 731727.5900000001, 727952.99, 724170.53, 720380.19, 716581.95, 712

# Total interest paid

In [20]:
print(sum(list(schedule1.Interest)))

-271766.9199999999
